In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/Cida/Documentos/utilities-water/data/guariroba/query_result_2025-01-09T17_21_18.465681Z.csv')

In [3]:
import re
import pandas as pd

def analisar_ceps_do_dataframe(df, zipcode):
    """
    Analisa uma coluna de CEPs em um DataFrame, removendo hífens,
    identificando CEPs inválidos (incompletos, com caracteres não numéricos
    ou com menos de 8 dígitos, ou com todos os dígitos iguais a zero) 
    e retornando dois DataFrames: um contendo
    as linhas com CEPs inválidos e outro com as linhas com CEPs válidos.

    Args:
        df: O DataFrame do pandas contendo a coluna de CEPs.
        zipcode: O nome da coluna no DataFrame que contém os CEPs.

    Returns:
        Um tuple contendo dois DataFrames:
            - O primeiro DataFrame contém apenas as linhas com CEPs inválidos.
            - O segundo DataFrame contém apenas as linhas com CEPs válidos.
    """

    # Criar a coluna 'CEP_Invalido' inicializada com False
    df['CEP_Invalido'] = False

    for index, row in df.iterrows():
        cep = row[zipcode]

        # Converter para string se não for
        if not isinstance(cep, str):
            cep = str(cep)

        # Remover hífens e pontos
        cep_sem_hifen = cep.replace("-", "").replace(".", "")

        # Verificar se contém apenas números e tem 8 dígitos
        if not re.match(r"^\d{8}$", cep_sem_hifen):
            df.loc[index, 'CEP_Invalido'] = True
        # Verificar se todos os dígitos são 0
        elif cep_sem_hifen == "00000000":
            df.loc[index, 'CEP_Invalido'] = True

    # Criar DataFrames contendo apenas os CEPs inválidos e válidos
    df_invalidos = df[df['CEP_Invalido'] == True].copy()
    df_validos = df[df['CEP_Invalido'] == False].copy()

    # Remover a coluna auxiliar 'CEP_Invalido' dos DataFrames resultantes
    df_invalidos = df_invalidos.drop(columns=['CEP_Invalido'], errors='ignore')
    df_validos = df_validos.drop(columns=['CEP_Invalido'], errors='ignore')

    return df_invalidos, df_validos

# Exemplo de uso:
# Suponha que você tenha um DataFrame chamado 'df' com uma coluna 'zipcode' contendo os CEPs.

# Analisar a coluna de CEPs
df_invalidos, df_validos = analisar_ceps_do_dataframe(df.copy(), "zipcode")  # Passa uma cópia para não modificar o original


In [4]:
df_invalidos['zipcode'].unique()

array([nan, '79070', '79032', 'string', '06835', '79060', '79072',
       '79077', '79045', '79080', '79043', '79079', '79105', '27515',
       '05009', '33496', '01551', '91360', '24030', '03146', '24070',
       '22723', '03142', '04501', '04763', '01306', '01303', '05717',
       '01304', '90010', '03348', '04733', '05444', '03069', '04624',
       '05049', '03186', '227445271', '888330030', '0000000', '00000-000',
       '3125010', '28', '11055', 'Caminho MARIA DA GRACA, 57',
       '581400560', '000000', '00000000', '0708020', 'G2K 0P3', 'G2B 2E4',
       '584100003', '040070000', '0470000', '0', '400086', '000000000',
       '7800000', '01552'], dtype=object)

In [5]:
df_invalidos['device_id'].nunique()

1517

In [ ]:
cep_df=pd.read_csv('/home/Cida/Desktop/Cep_estados.csv')

In [ ]:
import pandas as pd

def comparar_por_org_e_cidade(df_validos, cep_df, zipcode, Inicio, Fim, city, Região, org_name):
    df_validos = df_validos.copy()
    cep_df = cep_df.copy()

# Verifique as colunas para diagnosticar o erro
  #  print("Colunas do df_validos:", df_validos.columns)
  #  print("Colunas do cep_df:", cep_df.columns)    

    # Preparação das colunas
    df_validos['cep5'] = df_validos[zipcode].astype(str).str.zfill(8).str[:5]
    
    # Usar cidade de df_validos
    df_validos['cidade_lower'] = df_validos[city].astype(str).str.lower().str.strip()

    # Como a coluna de cidade não está presente em cep_df, omitimos a verificação de cidade
    cep_df['inicio5'] = cep_df[Inicio].astype(str).str.zfill(8).str[:5]
    cep_df['final5'] = cep_df[Fim].astype(str).str.zfill(8).str[:5]
    cep_df['regiao'] = cep_df[Região]

    regioes = []
    compatibilidades = []

    for idx, row in df_validos.iterrows():
        cep5 = row['cep5']
        cidade = row['cidade_lower']

        # Filtrar cep_df para encontrar a região correspondente ao CEP
        # e verificar a compatibilidade com a cidade
        # Aqui, assumimos que a cidade não é usada para filtrar, pois cep_df não contém essa informação
        faixas = cep_df[
            (cep_df['inicio5'] <= cep5) &
            (cep_df['final5'] >= cep5)
        ]

        if not faixas.empty:
            regioes.append(faixas.iloc[0]['regiao'])
            compatibilidades.append('Compatível')
        else:
            regioes.append(None)
            compatibilidades.append('Incompatível')

    df_validos['regiao'] = regioes
    df_validos['compatibilidade'] = compatibilidades

    return df_validos


# Exemplo de uso
df_resultado = comparar_por_org_e_cidade(
    df_validos.copy(),
    cep_df.copy(),
    'zipcode',     # Coluna com os CEPs no df_validos
    'Inicio',      # Coluna com início do intervalo de CEP no cep_df
    'Fim',         # Coluna com fim do intervalo de CEP no cep_df
    'city',        # Coluna com nome da cidade no df_validos
    'Região',      # Coluna com nome da região no cep_df
    'org_name'     # Nome da coluna que representa a organização
)

In [8]:
contagem_compatibilidade = df_resultado['compatibilidade'].value_counts()
print("\nContagem de Compatibilidade:")
print(contagem_compatibilidade)


Contagem de Compatibilidade:
compatibilidade
Compatível      23964
Incompatível      189
Name: count, dtype: int64


In [12]:
# Filtrando as linhas onde a coluna 'compatibilidade' é 'incompatível'
df_incompativeis = df_resultado[df_resultado['compatibilidade'] == 'Incompatível']

# Imprimindo as linhas filtradas
df_incompativeis

,device_id,activated_at,channels,activated_on_org_id,org_name,country_code,city,state,zipcode,lat,long,cep5,cidade_lower,regiao,compatibilidade
15738,836271019,"24 Sep, 2024, 18:59",1,297,Energisa,BR,Porto Velho,Rondônia,76805-846,-8.77,-63.90,76805,porto velho,None,Incompatível
16036,3007694189,"1 Oct, 2024, 18:32",1,400,TECMETRA,BR,Aracaju,Sergipe,.49001377,-11.08,-37.11,.4900,aracaju,None,Incompatível
16081,836264199,"2 Oct, 2024, 14:01",1,297,Energisa,BR,Porto Velho,Rondônia,76822-592,-8.73,-63.88,76822,porto velho,None,Incompatível
16083,836271011,"2 Oct, 2024, 14:25",1,297,Energisa,BR,Porto Velho,Rondônia,76822-592,-8.73,-63.88,76822,porto velho,None,Incompatível
16647,836264141,"5 Oct, 2024, 13:19",1,297,Energisa,BR,Porto Velho,Rondônia,76823-016,-8.71,-63.88,76823,porto velho,None,Incompatível
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24701,836264192,"22 Oct, 2024, 14:44",1,297,Energisa,BR,Porto Velho,Rondônia,76808-132,-8.80,-63.88,76808,porto velho,None,Incompatível
24702,836263589,"9 Dec, 2024, 19:02",1,514,Roraima Energia S.A,BR,Bonfim,Roraima,69390-000,2.62,-60.28,69390,bonfim,None,Incompatível
24710,836263420,"9 Dec, 2024, 20:31",1,297,Energisa,BR,Porto Velho,Rondônia,76810-476,-8.80,-63.89,76810,porto velho,None,Incompatível
24886,836264212,"17 Oct, 2024, 16:40",1,297,Energisa,NaN,Porto Velho,Rondônia,76824084,-8.74,-63.86,76824,porto velho,None,Incompatível
